In [91]:
from glob import glob
from PIL import Image
import pandas as pd
import numpy as np
import re
import math
from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense
from keras.models import Model

In [66]:
# 參數
# 決定要把讀進來的照片resize的程度
img_rows = 600
img_cols = 300
channels = 3
img_size = (img_rows, img_cols, channels)
img_path_list = glob('../0806/land_streetView/*degree/新北市板橋區華東街*.jpg')

In [34]:
import os 
os.path.isfile(img_path_list[0])

True

### Load label and picture

In [2]:
street_df = pd.read_csv('../0703/all_geo.csv')
land_df = pd.read_csv('../0703/all_lat.csv')

c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# 切割市 & 區
re.split('市', re.split('區', street_df['address'][0])[0])

['新北', '板橋']

In [6]:
street_df[:5000].head()

,address,lat,lng
0,新北市板橋區華東街271~300號,25.001936,121.452653
1,新北市土城區永豐路121~150號,24.984078,121.466060
2,新北市板橋區新崑路91~120號,24.999177,121.432317
3,新北市土城區延和路76巷1~30號,24.989084,121.466463
4,新北市板橋區中山路一段181~210號,25.013187,121.467601


In [14]:
# 找哪個區目前有的照片最多 (street_df 前5000筆)
for index, row in street_df[:5000].iterrows():
    pattern = re.split('市', re.split('區', row['address'])[0])[1]
    street_df.loc[index, 'section'] = pattern

In [35]:
# 查看哪區最多
street_df['section'].value_counts()

板橋    550
中和    536
新莊    519
三重    422
新店    391
淡水    356
汐止    320
土城    282
永和    266
蘆洲    228
林口    222
樹林    205
三峽    144
鶯歌    115
五股    114
泰山    104
瑞芳     50
八里     45
三芝     43
深坑     36
萬里     20
金山     19
貢寮      5
石門      3
雙溪      3
烏來      1
石碇      1
Name: section, dtype: int64

In [37]:
# 印出為板橋區的地址
street_df.loc[street_df['section'] == '板橋', 'address'].head()

0      新北市板橋區華東街271~300號
2       新北市板橋區新崑路91~120號
4    新北市板橋區中山路一段181~210號
5    新北市板橋區三民路一段151~180號
7        新北市板橋區廣和街61~90號
Name: address, dtype: object

In [39]:
for i, j in street_df.loc[street_df['section'] == '板橋', 'address'].head().iteritems():
    print(i)

0
2
4
5
7


`新北市板橋區華東街271~300號` 兩年交易110次
> https://www.leju.com.tw/page_search_result?oid=Lb7d85292713c4

In [79]:
test = pd.DataFrame()

In [80]:
# 找出地址符合street前5000的，並幫street標上售價
for i, item in street_df[:5000].loc[street_df[:5000]['section'] == '板橋'].iterrows():
#     print(item['address'])
#     print(len(land_df.loc[land_df['土地區段位置/建物區段門牌'] == item['address']]['總價元']))
    for j, price in land_df.loc[land_df['土地區段位置/建物區段門牌'] == item['address']].iterrows():
        test.loc[len(test), 'address'] = item['address']
        test.loc[len(test)-1, 'price'] = price['總價元']

In [81]:
test.head()

,address,price
0,新北市板橋區華東街271~300號,15000000.0
1,新北市板橋區華東街271~300號,32680000.0
2,新北市板橋區華東街271~300號,22770000.0
3,新北市板橋區華東街271~300號,17100000.0
4,新北市板橋區華東街271~300號,15420000.0


In [ ]:
# load image
def load_data(img_res, path_list):
    def imread(path, img_res):
        # 用 PIL讀圖片
        return np.array(Image.open(path).convert('RGB').resize(img_res, Image.ANTIALIAS)) / 255
        
    img_set = np.zeros((len(path_list), ) + img_res)
    # 根據檔案名稱分次讀圖近來
    for idx, rand_img_path in enumerate(path_list):
        img_set[idx] = imread(rand_img_path, img_res[:2])[:,:,:3]
            
    return img_set

In [ ]:
# 根據test DF處理圖片的形狀(完整的path_list, 第一維的size要跟label一致)

In [118]:
train_pic = []
label = []

In [119]:
# 判斷每個地址有幾張圖
for m, data in street_df[:5000].loc[street_df[:5000]['section'] == '板橋'].iterrows():
    backup_num = []
    try:
        address_num = re.findall('\d+', data['address'])
        if(len(re.findall('\d+', data['address'])) > 2):
            # 存大於2個的數值(有巷/弄)
            backup_num = address_num
            # 取倒數兩個
            address_num = address_num[-2:]

        size = math.ceil((int(address_num[1]) - int(address_num[0]))/10)
        address_split = re.split('\d+', data['address'])
        address_split.remove('~')
        address_split.remove('號')
        address_template = ''.join(address_split)
        if(len(backup_num) == 3):
            #有巷
            address_split[0] = address_split[0] + backup_num[0]
            address_template = ''.join(address_split)
        if(len(backup_num) == 4):
            # 有巷也有弄
            address_split[0] = address_split[0] + backup_num[0]
            address_split[1] = address_split[1] + backup_num[1]
            address_template = ''.join(address_split)

        img_path_list = glob('../0806/land_streetView/*degree/' + address_template + '*.jpg')
        match_list = [] # 記錄有幾張照片符合規則
        price_list = [] # 此地址有多少種售價
        for img in img_path_list:
            pattern = re.compile(address_template + '\d+號')
            if pattern.search(img):
                match_list.append(img)
                # 紀錄單一地址對應到哪些照片路徑

        for w, data in test.iterrows():
            pattern = re.compile(address_template + '\d+\~\d+號')
            if pattern.search(data['address']):
                price_list.append(data['price'])

        # 處理label, 根據有幾張圖，決定要給幾次一樣的label
        # 一個地址有多少種prcie
        for k in range(0, len(price_list)):
            # 每種售價根據照片數目填上label
            for index in range(0, len(match_list)):
                label.append(price_list[k])
        
    except Exception as e:
        print(e)
        continue

# list to numpy array
label = np.array(label)

list index out of range
list index out of range


In [120]:
label.shape

(608281,)

In [121]:
np.save('label_index5000_all_section', label)

In [122]:
np.load('label_index5000_all_section.npy').shape

(608281,)

### CNN model

In [ ]:
model_CNN = Sequential()
model_CNN.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model_CNN.add(Conv2D(64, (3, 3), activation='relu'))
model_CNN.add(MaxPooling2D(pool_size=(2, 2)))
model_CNN.add(Dropout(0.25))
model_CNN.add(Flatten())

model_CNN.add(Dense(128, activation='relu'))
model_CNN.add(Dropout(0.5))
# 預測價格
model_CNN.add(Dense(1, activation='relu'))

model_CNN.summary()

In [ ]:
model_CNN.compile(loss='mean_absolute_percentage_error', optimizer='adam')
train_history = model_CNN.fit(x=x_Train_norm, y=label, validation_split=0.3, epochs=10, batch_size=100, verbose=1)

### Multimodal
> https://keras.io/visualization/

In [ ]:
# Define two input layers
image_input = Input((32, 32, 3))
vector_input = Input((6,))

# Convolution + Flatten for the image
conv_layer = Conv2D(32, (3,3))(image_input)
flat_layer = Flatten()(conv_layer)

# Concatenate the convolutional features and the vector input
concat_layer= Concatenate()([vector_input, flat_layer])
output = Dense(3)(concat_layer)

# define a model with a list of two inputs
model = Model(inputs=[image_input, vector_input], outputs=output)